In [215]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [216]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')
from sklearn.metrics.pairwise import cosine_similarity


In [217]:
movies = pd.read_csv('../input/tmdbdataset/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdbdataset/tmdb_5000_credits.csv')

In [218]:
credits.head(1)


In [219]:
movies = movies.merge(credits, on='title')

> **#genres
> #id
> #keywords
> #title
> #overview
> #cast
> > #crew**


In [220]:
movies.head(1)

In [221]:
movies['original_language'].value_counts()

In [222]:
movies = movies[['movie_id', 'title', 'overview', 'genres' ,'cast', 'crew', 'keywords']]
movies.head(1)

In [223]:
movies.dropna(inplace = True)

In [224]:
movies.isnull().sum()

In [225]:
movies.duplicated().sum()

In [226]:
movies.head(1)

In [227]:
import ast

In [228]:
def convert1(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])  
    return L

In [229]:
movies['genres'] = movies['genres'].apply(convert1)
movies.head(1)

In [230]:
movies['cast'] = movies['cast'].apply(convert2)

In [231]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
            
    return L

In [232]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [233]:
def convert4(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])  
    return L

In [234]:
movies['keywords'] = movies['keywords'].apply(convert4)

In [235]:
movies.head(1)

In [236]:
movies.head(1)

In [237]:
movies['crew']

In [238]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [239]:
movies.head(1)

In [240]:
movies['genres'] =  movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])
movies['genres']

In [241]:
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])

In [242]:
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

In [243]:
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

In [244]:
movies.head()

In [245]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew'] + movies['keywords']

In [246]:
movies['tags']

In [247]:
new_df = movies[['movie_id', 'title', 'tags']]

In [248]:
new_df

In [249]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [250]:
new_df['tags']

In [251]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [252]:
new_df

In [253]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [254]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [255]:
new_df['tags'] = new_df['tags'].apply(stem)

In [256]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [257]:
vectors[0]


In [258]:
new_df['tags'] = new_df['tags'].apply(stem)

In [259]:
cv.get_feature_names()

In [260]:
similarity = cosine_similarity(vectors)

add Codeadd Markdown
****sorted(list(enumerate(similarity[0])),reverse = True, key = lambda x: x[1])[1:6]

In [261]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances =similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True, key = lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [262]:
recommend('Batman Begins')

In [263]:
import pickle

In [264]:
movie_file = "movie.pickle"
with open(movie_file,'wb') as f:
    pickle.dump(new_df, f)

pickle.dump(new_df,open('movies.pkl', 'wb'))